# one_party_data_preprocessing

## 事前準備
此篇 tutorial 為單 party 可執行的範例

### 設定資料路徑 & 參數

In [25]:
import os
guest = 9999
data_base = "/data/projects/fate/"

dense_data = {"name": "one_party_data_preprocessing", "namespace": f"experiment"}
dense_data_dir = os.path.join(data_base, "persistence/data/one_party_data_preprocessing.csv")

### 製作 Mimic Data

In [2]:
import pandas as pd
data = {
    "id":[0, 1, 2, 3, 4, 5, 6],
    "target":["0", "1", "0", "0", "1", "1", "0"],
    "x1":[0.1, 1.0, 0.2, None, 0.4, 0.5, 0.1],
    "x2":["A", "B", "B", "A", "C", "A", "A"],
    "x3":[0, 1, 0, None, 0, None, 1],
}
pd.DataFrame(data).to_csv(dense_data_dir, index=False)

### 缺失值 & 資料

In [3]:
dense_df = pd.read_csv(dense_data_dir)
print(dense_df.isna().mean())
print(dense_df.head(5))

id        0.000000
target    0.000000
x1        0.142857
x2        0.000000
x3        0.285714
dtype: float64
   id  target   x1 x2   x3
0   0       0  0.1  A  0.0
1   1       1  1.0  B  1.0
2   2       0  0.2  B  0.0
3   3       0  NaN  A  NaN
4   4       1  0.4  C  0.0


### 上傳資料到 pod 裡

In [26]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=9999).set_roles(guest=9999)
partition = 4


pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition,               # data info
                               )               
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-06-05 09:33:56.245 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306050933561032700

2023-06-05 09:33:56.252 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


m2023-06-05 09:33:57.267 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-05 09:33:57.268 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:01
2023-06-05 09:33:58.282 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-06-05 09:33:59.298 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-06-05 09:34:00.327 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-06-05 09:34:01.343 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-06-05 09:34:02.358 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-06-05 

### 建構 Pipeline

In [48]:
from pipeline.component import Reader

## Reader 讀取 Pod 裡的資料
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=9999).component_param(
    table={"name": "one_party_data_preprocessing", "namespace": "experiment"})

## Data transform 

`DataTransform` 使用 `component_param(params)` 設定參數, 可一次完成以下三項功能

#### 設定目標變數
1. `with_label=True` ( 因為 label 有在這個 party 的資料表裡 )
2. `label_name='target'` ( default `label_name = 'y'`, 要設定成本資料表的目標變數欄位名`'target'`, 否則會報錯 )
3. `label_type='int'` ( 轉換 label 的資料型態, 配合任務做選擇, 這邊假設是 Binary Classification )

#### 填補缺失值

1. 自動填值 -> `missing_fill=True, missing_fill_method='min' or 'max' or 'mean'`
2. 指定填值 -> `missing_fill=True, missing_fill_method='designated', default_value= any or [list of feature len]`
3. 定義缺失值 -> `missing_impute=[ "", "none", "null", "na", "None", NaN ]` 

#### 替換離群值

1. 自動填值 -> `outlier_replace=True, outlier_replace_method='min' or 'max' or 'mean'`
2. 指定填值 -> `outlier_replace=True, outlier_replace_method='designated', default_value = any or [list of feature len]`
3. 定義離群值 -> `outlier_replace_value=[ "", "none", "null", "na", "None", NaN ]` 

#### 其他重要參數

1. input 資料型態 `data_type="int" or "int64" or "float" or "float64" or "str" or "long"`
2. input 例外資料型態 `exclusive_data_type_fid_map={"colname":"int" or "int64" or "float" or "float64" or "str" or "long"}`

In [49]:
from pipeline.component import DataTransform

data_transform_0 = DataTransform(name="data_transform_0")
# set data transform parameter
data_transform_0.get_party_instance(role='guest', party_id=9999).component_param(
    with_label=True, label_name='target', label_type="int",
    missing_fill=True, missing_fill_method="designated", default_value=[0.0, "A", "1.0"],
    outlier_replace=True, outlier_replace_method="designated", outlier_replace_value=[0.0, "A", "1.0"],
    data_type="float", exclusive_data_type={"x2":"str", "x3":"str"}
)

## One hot encode

`OneHotEncoder` 使用 `component_param(params)` 設定參數

In [53]:
from pipeline.component import OneHotEncoder

onehot_encoder_0 = OneHotEncoder(name="onehot_encoder_0")

onehot_encoder_0.get_party_instance(role='guest', party_id=9999).component_param(
    transform_col_indexes=[3, 4], transform_col_names=["x2", "x3"]
)

## 執行整串pipeline

執行 `pipeline.compile()` & `pipeline.fit()` 看看資料轉換的成效

In [54]:
from pipeline.backend.pipeline import PipeLine
from pipeline.interface import Data

pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=9999) \
        .set_roles(guest=9999)

pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(onehot_encoder_0, data=Data(data=data_transform_0.output.data))
pipeline.compile()
pipeline.fit()

2023-06-05 09:50:17.739 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306050950175839430

2023-06-05 09:50:17.746 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-06-05 09:50:18.756 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-06-05 09:50:19.770 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-05 09:50:19.772 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-06-05 09:50:20.787 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-06-05 09:50:21.802 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-06-05 09:50:22.816 | INFO     | pipel

2023-06-05 09:50:54.648 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:36
2023-06-05 09:50:55.662 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:37
2023-06-05 09:50:56.678 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-06-05 09:50:57.694 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
2023-06-05 09:50:58.710 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:40
2023-06-05 09:50:59.724 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:41
2023-06-05 09:51:00.739 | INFO     | pipeline.utils.invoker.job_

#### 查看模塊狀況

用 `pipeline.get_component('module_name').get_summary()` 來看結果是否符合想像

In [62]:
import json
data_base = "/data/projects/fate/"
metadata_saved_dir = os.path.join(data_base, "persistence/metadata/one_party_data_transform_tutorial.json")
metedata = json.dumps(pipeline.get_component('data_transform_0').get_summary(), indent=4)
#data = pipeline.get_component('data_transform_0').get_output_data(limit=10)

with open(metadata_saved_dir, "w") as json_file:
    json_file.write(metedata)
                                  
print(f"Write in metadata_saved_dir : {metadata_saved_dir} \n {metedata}")

Write in metadata_saved_dir : /data/projects/fate/persistence/metadata/one_party_data_transform_tutorial.json 
 {
    "missing_fill_info": {
        "missing_impute_rate": {
            "x1": 0.14285714285714285,
            "x2": 0.0,
            "x3": 0.2857142857142857
        },
        "missing_impute_value": {
            "x1": 0.0,
            "x2": "A",
            "x3": "1.0"
        },
        "missing_value": [
            "",
            "none",
            "null",
            "na",
            "None",
            NaN
        ]
    },
    "outlier_replace_rate": {
        "outlier_replace_rate": {
            "x1": 0.0,
            "x2": 0.0,
            "x3": 0.0
        },
        "outlier_replace_value": {
            "x1": 0.0,
            "x2": "A",
            "x3": "1.0"
        },
        "outlier_value": [
            "",
            "none",
            "null",
            "na",
            "None",
            NaN
        ]
    }
}


#### 查看資料狀況

用 `pipeline.get_component('module_name').get_output_data(limits=n_data)` 來看結果是否符合想像

In [59]:
pipeline.get_component('reader_0').get_output_data(limits=10)

,id,target,x1,x2,x3
0,0,0,0.1,A,0.0
1,1,1,1.0,B,1.0
2,2,0,0.2,B,0.0
3,3,0,,A,
4,4,1,0.4,C,0.0
5,5,1,0.5,A,
6,6,0,0.1,A,1.0


In [60]:
pipeline.get_component('data_transform_0').get_output_data(limits=10)

,id,target,x1,x2,x3
0,5,1,0.5,A,1.0
1,2,0,0.2,B,0.0
2,0,0,0.1,A,0.0
3,1,1,1.0,B,1.0
4,6,0,0.1,A,1.0
5,3,0,0.0,A,1.0
6,4,1,0.4,C,0.0


In [61]:
pipeline.get_component('onehot_encoder_0').get_output_data(limits=10)

,id,target,x1,x2_B,x2_C,x2_A,x3_0.0,x3_1.0
0,5,1,0.5,0.0,0.0,1.0,0.0,1.0
1,2,0,0.2,1.0,0.0,0.0,1.0,0.0
2,0,0,0.1,0.0,0.0,1.0,1.0,0.0
3,1,1,1.0,1.0,0.0,0.0,0.0,1.0
4,4,1,0.4,0.0,1.0,0.0,1.0,0.0
5,6,0,0.1,0.0,0.0,1.0,0.0,1.0
6,3,0,0.0,0.0,0.0,1.0,0.0,1.0
